In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("crftag1000.tsv", encoding="latin1")

In [2]:
data = data.fillna(method="ffill")


In [3]:
data.tail(5)

,Sentence #,Word,POS,Tag
92764,Sentence: 1000,good,JJ,o
92765,Sentence: 1000,practice,NN,o
92766,Sentence: 1000,is,VBZ,o
92767,Sentence: 1000,welcome,JJ,o
92768,Sentence: 1000,.,.,o


In [4]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words

10044

In [5]:
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags

22

In [6]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [7]:
getter = SentenceGetter(data)

In [9]:
sent = getter.get_next()
print(sent)

[('On', 'IN', 'o'), ('one', 'CD', 'o'), ('Linux', 'NNP', 'OS&SystemKernels'), ('Server', 'NNP', 'o'), ('running', 'VBG', 'o'), ('Apache', 'NNP', 'o'), ('and', 'CC', 'o'), ('PHP', 'NNP', 'Web development'), ('5', 'CD', 'o'), (',', ',', 'o'), ('we', 'PRP', 'o'), ('got', 'VBD', 'o'), ('multiple', 'JJ', 'o'), ('Virtual', 'NNP', 'o'), ('Hosts', 'VBZ', 'o'), ('with', 'IN', 'o'), ('separate', 'JJ', 'o'), ('log', 'NN', 'o'), ('files', 'NNS', 'o'), ('and', 'CC', 'o'), ('everything', 'NN', 'o'), ('.', '.', 'o'), ('The', 'DT', 'o'), ('only', 'JJ', 'o'), ('thing', 'NN', 'o'), ('we', 'PRP', 'o'), ('can', 'MD', 'o'), ('not', 'RB', 'o'), ('seem', 'VB', 'o'), ('to', 'TO', 'o'), ('separate', 'VB', 'o'), ('between', 'IN', 'o'), ('virtual', 'JJ', 'o'), ('hosts', 'NNS', 'o'), ('is', 'VBZ', 'o'), ('the', 'DT', 'o'), ('php', 'NN', 'Web development'), ('.', '.', 'o'), ('Overriding', 'VBG', 'o'), ('this', 'DT', 'o'), ('setting', 'NN', 'o'), ('in', 'IN', 'o'), ('the', 'DT', 'o'), ('of', 'IN', 'o'), ('the', 'DT

In [10]:
sentences = getter.sentences


In [11]:
max_len = 150
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [12]:
tag2idx["o"]

5

In [13]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

C:\Users\ALK4KOR\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)


In [15]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]


In [16]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["o"])


In [17]:
from keras.utils import to_categorical


In [18]:
y = [to_categorical(i, num_classes=n_tags) for i in y]


In [19]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)


In [20]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

In [21]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [22]:
model = Model(input, out)

In [23]:
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])


In [24]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 20)           200900    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 100)          28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 150, 50)           5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 150, 22)           1650      
Total params: 236,000
Trainable params: 236,000
Non-trainable params: 0
_________________________________________________________________


In [25]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=50,
                    validation_split=0.1, verbose=1)

Train on 810 samples, validate on 90 samples
Epoch 1/50
810/810 [==============================] - 11s 13ms/step - loss: 9.6161 - acc: 0.8625 - val_loss: 7.2709 - val_acc: 0.9716
Epoch 2/50
810/810 [==============================] - 9s 11ms/step - loss: 8.9195 - acc: 0.9786 - val_loss: 7.1948 - val_acc: 0.9716
Epoch 3/50
810/810 [==============================] - 9s 11ms/step - loss: 8.8675 - acc: 0.9786 - val_loss: 7.1611 - val_acc: 0.9716
Epoch 4/50
810/810 [==============================] - 10s 12ms/step - loss: 8.8389 - acc: 0.9788 - val_loss: 7.1376 - val_acc: 0.9716
Epoch 5/50
810/810 [==============================] - 9s 11ms/step - loss: 8.8147 - acc: 0.9786 - val_loss: 7.1133 - val_acc: 0.9716
Epoch 6/50
810/810 [==============================] - 9s 11ms/step - loss: 8.7883 - acc: 0.9785 - val_loss: 7.0853 - val_acc: 0.9716
Epoch 7/50
810/810 [==============================] - 9s 11ms/step - loss: 8.7599 - acc: 0.9785 - val_loss: 7.0561 - val_acc: 0.9716
Epoch 8/50
810/810 [==